In [ ]:
from binance.client import Client
import pandas as pd

# Initialiser le client Binance (pas besoin de clé API pour les données publiques)
client = Client()

def get_5min_klines(symbol, days=1):
    """
    Récupère les données de bougies (klines) pour un symbole donné, sur des intervalles de 5 minutes, pour une journée.

    :param symbol: Paire de trading (ex: "BTCUSDT")
    :param days: Nombre de jours (1 par défaut)
    :return: DataFrame pandas avec les données
    """
    # Récupérer les klines (bougies) sur 5 minutes
    klines = client.get_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_5MINUTE, limit=288 * days)

    # Convertir les données en DataFrame pandas
    columns = [
        'Open time', 'Open', 'High', 'Low', 'Close', 'Volume',
        'Close time', 'Quote asset volume', 'Number of trades',
        'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'
    ]
    df = pd.DataFrame(klines, columns=columns)

    # Convertir les colonnes numériques en float
    numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote asset volume']
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col])

    # Convertir le timestamp en datetime
    df['Open time'] = pd.to_datetime(df['Open time'], unit='ms')

    return df

# Exemple d'utilisation pour BTC/USDT sur 1 jour
symbol = "BTCUSDT"
data = get_5min_klines(symbol, days=1)

# Afficher les 5 premières lignes
print(data[['Open time', 'Open', 'High', 'Low', 'Close', 'Volume']].head())


In [ ]:

import ccxt
import pandas as pd
import numpy as np
import pandas_ta as ta
from datetime import datetime, timedelta, timezone


exchange = ccxt.binance()
symbols = ["BTC/USDT", "ETH/USDT", "BNB/USDT", "XRP/USDT", "SOL/USDT"]


interval = "5m"
since = int((datetime.now(timezone.utc) - timedelta(days=30)).timestamp() * 1000)


dfs = []


for symbol in symbols:
    print(f"→ Téléchargement de {symbol} ...")
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=interval, since=since, limit=None)
    df = pd.DataFrame(ohlcv, columns=["timestamp","open","high","low","close","volume"])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms", utc=True)
    df.set_index("timestamp", inplace=True)


    # Features techniques
    df["return"] = df["close"].pct_change()
    df["log_return"] = (df["close"] / df["close"].shift(1)).apply(lambda x: pd.NA if x<=0 else np.log(x))
    df["volatility"] = df["log_return"].rolling(12).std()  # ~ 2h rolling vol
    df["rsi"] = ta.rsi(df["close"], length=14)
    macd = ta.macd(df["close"], fast=12, slow=26)
    df = pd.concat([df, macd], axis=1)
    bbands = ta.bbands(df["close"], length=20)
    df = pd.concat([df, bbands], axis=1)


    df["symbol"] = symbol
    dfs.append(df)


# Combine all cryptos
full_df = pd.concat(dfs)
full_df.reset_index(inplace=True)


# Construire un indice de marché global pondéré par volume
pivot_close = full_df.pivot_table(index="timestamp", columns="symbol", values="close")
pivot_vol = full_df.pivot_table(index="timestamp", columns="symbol", values="volume")
weights = pivot_vol.div(pivot_vol.sum(axis=1), axis=0)
weighted_returns = pivot_close.pct_change().mul(weights)
full_df_market = pd.DataFrame()
full_df_market["market_return"] = weighted_returns.sum(axis=1)
full_df_market["total_volume"] = pivot_vol.sum(axis=1)
full_df_market["btc_dominance"] = pivot_vol["BTC/USDT"] / pivot_vol.sum(axis=1)


# Merge dataset agrégé
final_dataset = full_df.merge(full_df_market, on="timestamp", how="left")


print("✅ Dataset final :", final_dataset.shape)
print(final_dataset.head())
